In [2]:
import os
import numpy as np

def parse_smesh(file_path):
    """Parse smesh file, skipping first 5 lines and last line, and ignoring the fourth column."""
    with open(file_path, 'r') as f:
        lines = f.readlines()[5:-1]  # Skip first 5 lines and last line
    
    # Convert to float while ignoring the fourth column
    data = np.array([list(map(float, line.split()[:3])) for line in lines])
    return data.reshape((17, 12, 3))

def load_data(base_path):
    """Load and process input geometries and final geometry results."""
    input_geometry_path = os.path.join(base_path, "Input_geometry")
    final_geometry_path = os.path.join(base_path, "final_geometry")
    
    input_data, temp_data, press_data = [], [], []
    output_data_80, output_data_140, labels = [], [], []
    
    # Load input geometries
    for run_id in range(1, 65):  # Runs are from run1 to run64
        run_dir = os.path.join(input_geometry_path, f"run{run_id}")
        smesh_files = [os.path.join(run_dir, f"smesh.1.{i}.txt") for i in range(1, 4)]
        
        if all(os.path.exists(f) for f in smesh_files):
            input_geometry = np.stack([parse_smesh(f) for f in smesh_files], axis=-1)  # Shape (17,12,3)
            input_data.append(input_geometry)
            
            # Extract temperature and pressure from file name
            temp_index = (run_id - 1) % 5 + 1  # Temperature index (1 to 5)
            press_index = ((run_id - 1) // 5) % 3 + 1  # Pressure index (1 to 3)
            temp_data.append(temp_index)
            press_data.append(press_index)
        else:
            print(f"Warning: Missing smesh files in {run_dir}. Skipping...")
    
    # Load final geometry results
    for run_id in range(1, 65):
        for tstep in [80, 140]:
            result_file = os.path.join(final_geometry_path, f"result_{tstep}_t{(run_id-1)%5+1}_p{((run_id-1)//5)%3+1}_{run_id}")
            
            if os.path.exists(result_file):
                deformation = np.loadtxt(result_file).reshape((17, 12, 3, 3))
                if tstep == 80:
                    output_data_80.append(deformation)
                else:
                    output_data_140.append(deformation)
            else:
                print(f"Warning: Missing result file {result_file}. Skipping...")
                if tstep == 80:
                    output_data_80.append(np.zeros((17, 12, 3)))
                else:
                    output_data_140.append(np.zeros((17, 12, 3)))
    
    return np.array(input_data), np.array(temp_data), np.array(press_data), np.array(output_data_80), np.array(output_data_140)

# Example usage
base_directory = "/home/exouser/ml_me5920/Assignment_2/DM/data"  # Change this to your actual directory
input_geometry, temp_array, press_array, output_80, output_140 = load_data(base_directory)

# Construct Ordered Pairs
ordered_pairs = []
for idx in range(len(input_geometry)):
    input_tuple = (input_geometry[idx], np.array([temp_array[idx]]), np.array([press_array[idx]]))
    output_80_pair = (input_tuple, output_80[idx])
    output_140_pair = (input_tuple, output_140[idx])
    ordered_pairs.append(output_80_pair)
    ordered_pairs.append(output_140_pair)

# Print example pair
example_pair = ordered_pairs[0]
print("Input shapes:", [x.shape for x in example_pair[0]], "Output shape:", example_pair[1].shape)


ValueError: cannot reshape array of size 1839 into shape (17,12,3,3)